## 43. 커스텀 컨테이너 타입은 collections.abc를 상속하라

시퀀스처럼 사용법이 간단한 클래스를 정의할 떄는 파이썬 내장 리스트 타입의 하위 클래스를 만들고 싶은 것이 당연하다.

예를 들어 멤버들의 빈도를 계산하는 메서드가 포함된 커스텀 리스트 타입이 필요하다고 가정하다.

In [1]:
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)
        
    def frequency(self):
        counts = {}
        for item in self:
            counts[item] = counts.get(item, 0) + 1
        return counts

FrequencyList를 리스트의 하위 클래스로 만듦으로써 리스트가 제공하는 모든 표준 함수를 사용할 수 있다.

In [2]:
foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'a', 'd'])
print('길이: ', len(foo))

길이:  7


In [3]:
foo.pop()

'd'

In [4]:
repr(foo)

"['a', 'b', 'a', 'c', 'b', 'a']"

In [5]:
foo.frequency()

{'a': 3, 'b': 2, 'c': 1}

이제 리스트처럼 느껴지면서 인덱싱이 가능한 객체를 제공하고 싶은데, 리스트의 하위 클래스로 만들고 싶지는 않다고 ㄱ자ㅓㅇ해보자.

예를 들어, 다음 이진 트리 클래스를 시퀀스(리스트나 튜플)의 의미 구조를 사용해 다룰 수 있는 클래스를 만들고 싶다.

In [6]:
class BinaryNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

In [7]:
bar = [1, 2, 3]
bar[0]

1

In [8]:
bar.__getitem__(0)

1

BinaryNode 클래스가 시퀀스처럼 작동하게 하려면 트리 노드를 깊이 우선 순회하는 커스텀 __getietm__ 메서드 구현을 제공하면 된다.

In [9]:
class IndexableNode(BinaryNode):
    def _traverse(self):
        if self.left is not None:
            yield from self.left._traverse()
        yield self
        if self.right is not None:
            yield from self.right._traverse()

    def __getitem__(self, index):
        for i, item in enumerate(self._traverse()):
            if i == index:
                return item.value
        raise IndexError(f'인덱스 범위 초과: {index}')

In [10]:
tree = IndexableNode(
    10,
    left=IndexableNode(
        5,
        left=IndexableNode(2),
        right=IndexableNode(
            6,
            right=IndexableNode(7))),
    right=IndexableNode(
        15,
        left=IndexableNode(11)))


       10
  5         15
2  6       11
     7

이 트리를 left나 right 애트리뷰트를 사용해 순회할 수도 있지만, 추가로 리스트처럼 접근할 수도 있다.

In [11]:
print('LRR:', tree.left.right.right.value)
print('인덱스 0:', tree[0])
print('인덱스 1:', tree[1])
print('11이 트리 안에 있나?', 11 in tree)
print('17이 트리 안에 있나?', 17 in tree)
print('트리:', list(tree))

LRR: 7
인덱스 0: 2
인덱스 1: 5
11이 트리 안에 있나? True
17이 트리 안에 있나? False
트리: [2, 5, 6, 7, 10, 11, 15]


문제는 getitem을 구현하는 것만으로는 리스트 인스턴스에서 기대할 수 있는 모든 시퀀스 의미 구조를 제공할 수 없다는데 있다.

In [12]:
len(tree)

TypeError: object of type 'IndexableNode' has no len()

len 내장함수를 구현해주자

커스텀 시퀀스 타입은 이 메서드를 꼭 구현해야한다

In [13]:
class SequenceNode(IndexableNode):
    def __len__(self):
        for count, _ in enumerate(self._traverse(), 1):
            pass
        return count

In [15]:
tree = SequenceNode(
    10,
    left=SequenceNode(
        5,
        left=SequenceNode(2),
        right=SequenceNode(
            6,
            right=SequenceNode(7))),
    right=SequenceNode(
        15,
        left=SequenceNode(11))
)

In [16]:
print('트리 길이:', len(tree))

트리 길이: 7


안타깝지만 getitem과 len으로 시퀀스가 되지는 않는다.

countㄴ타 index 메서드도 들어 있지 않다.

이런 어려움을 덜어주기 위해 내장 collections.abc 모듈 안에는 컨테이너 타입에 정의해야하는 전형적인 메서드를 모두 제공하는 추상기반 클래스 정의가 여러가지 들어 있다.

이런 추상 기반 클래스의 하위클래스를 만들고 필요한 메서드 구현을 잊어버리면, collections.abc 모듈이 실수한 부분을 알려준다.

In [17]:
from collections.abc import Sequence

class BadType(Sequence):
    pass

In [18]:
foo = BadType()

TypeError: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__

추상 기반 클래스가 요구하는 모든 메서드를 구현하면 index나 count와 같은 추가 메서드 구현을 거저 얻을 수 있다.

In [19]:
class BetterNode(SequenceNode, Sequence):
    pass

In [21]:
tree = BetterNode(
    10,
    left=BetterNode(
        5,
        left=BetterNode(2),
        right=BetterNode(
            6,
            right=BetterNode(7))),
    right=BetterNode(
        15,
        left=BetterNode(11))
)

In [22]:
print('7의 인덱스:', tree.index(7))
print('10의 개수:', tree.count(10))

7의 인덱스: 3
10의 개수: 1


## 기억해야 할 내용
- 간편하게 사용할 경우에는 파이썬 컨테이너 타입(리스트나 딕셔너리 등)을 직접 상속하라.
- 커스텀 컨테이너를 제대로 구현하려면 수많은 메서드를 구현해야 한다는 점에 주의하라
- 커스텀 컨테이너 타입이 collection.abc에 정의된 인터페이스를 상속하면 커스텀 컨테이너 타입이 정상적으로 작동하기 위해 필요한 인터페이스와 기능을 제대로 구현하도록 보장할 수 있다.